In [255]:
import yfinance as yf
# https://pypi.org/project/yfinance/
import pandas as pd
from datetime import datetime, timedelta, date
import time
# from reader import feed
from pandas.tseries.frequencies import to_offset
import numpy as np
# from pandas.tseries.offsets import BDay

In [256]:
tic = time.perf_counter()

today = date.today()
print("Today's date:", today)
days_back = 365*3
rows_drop = days_back-2
start_date = (date.today() - timedelta(days=days_back)).strftime('%Y-%m-%d')
start_date

Today's date: 2021-08-12


'2018-08-13'

In [257]:
# tickers = pd.read_csv("Input/tickers_input.csv")
# tickers = "SPY AAPL MSFT"
# tickers = ['AAPL','MSFT','AMD']
tickers = ['AAPL']
tickers

['AAPL']

In [258]:
# df = yf.download(tickers,start= start_date, end=date.today())
# df.head()

logic = {'Open'  : 'first',
         'High'  : 'max',
         'Low'   : 'min',
         'Close' : 'last',
         'Volume': 'sum'}

dfout = pd.DataFrame()

for ticker in tickers:
    df = yf.Ticker(ticker)
    df = df.history('3y')
#     df.to_csv('Output/df.csv')  # used to test is data is being collected correctly
    df.drop(columns=['Dividends','Stock Splits'],inplace=True)

    df1w = yf.download(ticker,period='3y',interval='1wk')
#     df1w = df.history('3y')
    df1w = df1w.dropna(how='any')
#     df1w.to_csv('Output/df1w.csv')  # used to test is data is being collected correctly
#     df.drop(columns=['Dividends','Stock Splits'],inplace=True)

    df3d = yf.download(ticker,period='3y',interval='1d')
#     df1w = df.history('3y')
    df3d = df3d.dropna(how='any')
#     df3d.to_csv('Output/df3d.csv')  # data checkpoint
    
    df3d = df.resample('3B').apply(logic)
#     df3d.index = df3d.index + to_offset('3B')
#     df3d.to_csv('Output/df3d.csv')  # data checkpoint
    
    df['conv_line'] = ((df.High.rolling(9).max())+(df.Low.rolling(9).min()))/2 #checked
    df['base_line'] = ((df.High.rolling(26).max())+(df.Low.rolling(26).min()))/2 #checked
    df['lead_a'] = ((df['conv_line']+df['base_line'])/2).shift(25) # checked
    df['lead_b'] = (((df.High.rolling(52).max())+(df.Low.rolling(52).min()))/2).shift(25) # checked
    df['lag_span'] = df.Close.shift(-25) #checked
    df['crit1'] = (df.conv_line>df.base_line).astype(int)  # conversion line above baseline, checked
    df['crit2'] = (df.lead_a.shift(-25)>df.lead_a.shift(-24)).astype(int)  # lead 1 moving up, checked
    df['crit3'] = (df.Close>df.lead_a).astype(int)  # price above Lead 1, checked
    df['crit4'] = (df.Close>df.lead_b).astype(int)  # price above Lead 2, checked
    df['crit5'] = (df.lead_a.shift(-25)>df.lead_b.shift(-25)).astype(int)  # lead 1 above lead 2, checked
    df['crit6'] = (((df.lag_span>df.High).shift(25)).fillna(0).astype(int)) # lag line above price action, checked
    df['crit7'] = (df.lead_b.shift(-25)>df.lead_b.shift(-24)).astype(int)  # lead 2 moving up, checked
    # df['test2'] = df.lead_a.shift(-25)-df.lead_a.shift(-24)  #  this code used to check crit2
    # df['test3'] = df.Close - df.lead_a # used to check crit3
    # df['test4'] = df.Close - df.lead_b # used to check crit4
    # df['test5'] = df.lead_a.shift(-25)-df.lead_b.shift(-25)  # test for crit5
    # df['test6'] = (df.lag_span - df.High).shift(25) # test for crit6
    # df['test7'] = df.lead_b.shift(-25)-df.lead_b.shift(-24)  #  this code used to check crit7
    df = df.drop(df.columns[[0,1,2,3,4,5,6,7,8,9]],axis=1, inplace=False)  #remove unnecessary columns
#     df.to_csv('Output/df.csv')  # data checkpoint

    df3d['conv_line'] = ((df3d.High.rolling(9).max())+(df3d.Low.rolling(9).min()))/2 #checked
    df3d['base_line'] = ((df3d.High.rolling(26).max())+(df3d.Low.rolling(26).min()))/2 #checked
    df3d['lead_a'] = ((df3d['conv_line']+df3d['base_line'])/2).shift(25) # checked
    df3d['lead_b'] = (((df3d.High.rolling(52).max())+(df3d.Low.rolling(52).min()))/2).shift(25) # checked
    df3d['lag_span'] = df3d.Close.shift(-25) #checked
    df3d['crit1'] = (df3d.conv_line>df3d.base_line).astype(int)  # conversion line above baseline, checked
    df3d['crit2'] = (df3d.lead_a.shift(-25)>df3d.lead_a.shift(-24)).astype(int)  # lead 1 moving up, checked
    df3d['crit3'] = (df3d.Close>df3d.lead_a).astype(int)  # price above Lead 1, checked
    df3d['crit4'] = (df3d.Close>df3d.lead_b).astype(int)  # price above Lead 2, checked
    df3d['crit5'] = (df3d.lead_a.shift(-25)>df3d.lead_b.shift(-25)).astype(int)  # lead 1 above lead 2, checked
    df3d['crit6'] = (((df3d.lag_span>df3d.High).shift(25)).fillna(0).astype(int)) # lag line above price action, checked
    df3d['crit7'] = (df3d.lead_b.shift(-25)>df3d.lead_b.shift(-24)).astype(int)  # lead 2 moving up, checked
    # df3d['test2'] = df3d.lead_a.shift(-25)-df3d.lead_a.shift(-24)  #  this code used to check crit2
    # df3d['test3'] = df3d.Close - df3d.lead_a # used to check crit3
    # df3d['test4'] = df3d.Close - df3d.lead_b # used to check crit4
    # df3d['test5'] = df3d.lead_a.shift(-25)-df3d.lead_b.shift(-25)  # test for crit5
    # df3d['test6'] = (df3d.lag_span - df3d.High).shift(25) # test for crit6
    # df3d['test7'] = df3d.lead_b.shift(-25)-df3d.lead_b.shift(-24)  #  this code used to check crit7
    df3d = df3d.drop(df3d.columns[[0,1,2,3,4,5,6,7,8,9]],axis=1, inplace=False)  #remove unnecessary columns
    df3d.to_csv('Output/df3d.csv')  # data checkpoint

    df1w['conv_line'] = ((df1w.High.rolling(9).max())+(df1w.Low.rolling(9).min()))/2 #checked
    df1w['base_line'] = ((df1w.High.rolling(26).max())+(df1w.Low.rolling(26).min()))/2 #checked
    df1w['lead_a'] = ((df1w['conv_line']+df1w['base_line'])/2).shift(25) # checked
    df1w['lead_b'] = (((df1w.High.rolling(52).max())+(df1w.Low.rolling(52).min()))/2).shift(25) # checked
    df1w['lag_span'] = df1w.Close.shift(-25) #checked
    df1w['crit1'] = (df1w.conv_line>df1w.base_line).astype(int)  # conversion line above baseline, checked
    df1w['crit2'] = (df1w.lead_a.shift(-25)>df1w.lead_a.shift(-24)).astype(int)  # lead 1 moving up, checked
    df1w['crit3'] = (df1w.Close>df1w.lead_a).astype(int)  # price above Lead 1, checked
    df1w['crit4'] = (df1w.Close>df1w.lead_b).astype(int)  # price above Lead 2, checked
    df1w['crit5'] = (df1w.lead_a.shift(-25)>df1w.lead_b.shift(-25)).astype(int)  # lead 1 above lead 2, checked
    df1w['crit6'] = (((df1w.lag_span>df1w.High).shift(25)).fillna(0).astype(int)) # lag line above price action, checked
    df1w['crit7'] = (df1w.lead_b.shift(-25)>df1w.lead_b.shift(-24)).astype(int)  # lead 2 moving up, checked
    # df1w['test2'] = df1w.lead_a.shift(-25)-df1w.lead_a.shift(-24)  #  this code used to check crit2
    # df1w['test3'] = df1w.Close - df1w.lead_a # used to check crit3
    # df1w['test4'] = df1w.Close - df1w.lead_b # used to check crit4
    # df1w['test5'] = df1w.lead_a.shift(-25)-df1w.lead_b.shift(-25)  # test for crit5
    # df1w['test6'] = (df1w.lag_span - df1w.High).shift(25) # test for crit6
    # df1w['test7'] = df1w.lead_b.shift(-25)-df1w.lead_b.shift(-24)  #  this code used to check crit7
    df1w = df1w.drop(df1w.columns[[0,1,2,3,4,5,6,7,8,9,10]],axis=1, inplace=False)  #remove unnecessary columns
#     df1w.to_csv('Output/df1w.csv')  # data checkpoint
    
    df.reset_index(inplace=True)
    df1w.reset_index(inplace=True)
#     df1w.to_csv('Output/df1w.csv')  # data checkpoint
    merged_df = pd.merge(df1w,\
                         df,\
                         left_on=[df1w['Date'].dt.year, df1w['Date'].dt.strftime('%U')],\
                         right_on=[df['Date'].dt.year, df['Date'].dt.strftime('%U')],\
                         how='right',suffixes=('_1W','_1D'),).drop(['key_0','key_1'],axis=1)

    merged_df=merged_df.apply(lambda x: x.fillna(method='ffill') if x.dtype.kind in 'biufc' else x.fillna(method='ffill'))

#     df = df.drop(df.columns[[0,1,2,3,4,5,6,7,8,9]],axis=1, inplace=False)  #remove unnecessary columns
#     df['total_pts'] = df.sum(axis=1)  # adds up the points
#     df = df.drop(df.columns[[0,1,2,3,4,5,6]],axis=1,inplace=False) #remove unnecessary columns
#     row_count = len(df.index) #get the row count of the dataframe
#     df = df.iloc[row_count-2: , :] #remove all rows except the last two
#     chg = df.total_pts.diff() #find difference between the last two rows
#     df['chg'] = chg #add the chg data to the dataframe
#     row_count = len(df.index) #get the row count of the dataframe
#     df = df.iloc[row_count-1: , :] # remove all but the last row
#     df['ticker'] = ticker # add ticker label column
#     df = df[['ticker','total_pts','chg']] # move ticker column to the front

#     dfout = dfout.append(df)
    dfout = dfout.append(merged_df)

dfout.to_csv('Output/check_file.csv')
# print(ticker)
# pd.set_option('display.max_rows',999)
dfout

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Date_1W,crit1_1W,crit2_1W,crit3_1W,crit4_1W,crit5_1W,crit6_1W,crit7_1W,Date_1D,crit1_1D,crit2_1D,crit3_1D,crit4_1D,crit5_1D,crit6_1D,crit7_1D
0,2018-08-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-08-13,0,0,0,0,0,0,0
1,2018-08-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-08-14,0,0,0,0,0,0,0
2,2018-08-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-08-15,0,0,0,0,0,0,0
3,2018-08-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-08-16,0,0,0,0,0,0,0
4,2018-08-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-08-17,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,2021-08-12,1.0,0.0,1.0,1.0,0.0,1.0,0.0,2021-08-10,1,0,1,1,0,1,0
756,2021-08-09,1.0,0.0,1.0,1.0,0.0,1.0,0.0,2021-08-11,1,0,1,1,0,1,0
757,2021-08-12,1.0,0.0,1.0,1.0,0.0,1.0,0.0,2021-08-11,1,0,1,1,0,1,0
758,2021-08-09,1.0,0.0,1.0,1.0,0.0,1.0,0.0,2021-08-12,1,0,1,1,0,1,0


In [259]:
toc = time.perf_counter()
print(f"Total processing time is {toc-tic:0.4f} seconds")
# count = len(tickers)

# # for i in count:
# df['50MA'] = tickers[0].Close.rolling(50).mean()
# df
# df.to_csv('check_file.csv')

Total processing time is 1.0914 seconds


In [260]:
df

,Date,crit1,crit2,crit3,crit4,crit5,crit6,crit7
0,2018-08-13,0,0,0,0,0,0,0
1,2018-08-14,0,0,0,0,0,0,0
2,2018-08-15,0,0,0,0,0,0,0
3,2018-08-16,0,0,0,0,0,0,0
4,2018-08-17,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
751,2021-08-06,1,0,1,1,0,1,0
752,2021-08-09,1,0,1,1,0,1,0
753,2021-08-10,1,0,1,1,0,1,0
754,2021-08-11,1,0,1,1,0,1,0
